# 用于高光谱图像分类的光谱-空间特征令牌化

<font color="green">
什么是令牌化:将卷积层提取到的光谱-空间特征转换为语义令牌的过程,核心是通过特定的特征矩阵和变换操作,将高维特征映射到具有语义信息的低维令牌序列中以便于transformer编码器处理
</font>

# 第一遍:
## 模型创新:
1.层级化光谱-空间特征提取模块<br>
2.高斯加权特征令牌化器<br>
3.transformer编码器与令牌融合<br>
4.轻量级网络设计<br>
## 模型优势:
1.分类性能优异:三个数据集上,总体精度(OA),平均精度(AA),Kappa系数均优于其他主流方法

<font color="green">
什么是OA:所有被正确分类的样本数占总样本数的比例<br>
什么是AA:分别计算背对每个类别分类的精度再取平均值<br>
</font>

<font color="red">
什么是kappa系数:考虑随机因素对分类结果的影响,kappa系数是一个-1到1之间的数,其值接近1说明分类结果与真实标签一致性极高,分类性能优异,接近于0说明分类效果接近于随机分类,接近于-1说明分类效果比随机分类效果差
</font>

2.计算效率高:训练时间和测试时间显著低于其他方法

3.对小样本和不平衡数据的鲁棒性

4.空间与光谱信息的高效融合

# 第二遍
## step1:数据读入


<img src="屏幕截图 2025-07-10 142714.png" width="300px">

- 输入一个$m \times n \times l$的HSI图像,经过主成分分析后将光谱维数由l降为b
- 将分析后的结果打碎成块传入3D卷积层假设卷积核个数为k0,就获得k0个新的特征图
- 将3D卷积后的特征图沿着光谱维的方向重排后传入2D卷积层假设2D卷积层有$k0'$个卷积核,则再生成k0'个特征图
- 需要注意的是,3d卷积的特征重排不是简单的图片叠加

- 数据流动
- 原始高光谱图像:$I \in \mathbb{R}^{m\times n \times l}$,$M \times n$是空间尺寸,l是光谱波段数
- 经过pca:$I \in \mathbb{R}^{m\times n \times b} $ b是自主选取的特征值最大的前b个波段
- 经过3d-patch-提取:$I \in \mathbb{R}^{s \times s \times b} $,其中,对于边缘像素,进行填充操作
- 3d卷积处理:假设每个3d核大小为$k_1 \times k_2 \times k_3 $则卷积后的的特征立方体大小为$(s-k_1+1) \times (s-k_2+1) \times (b-k_3+1)$
- 重排操作:对3d卷积层生成的特征方块进行重排,变为$(s-k_1+1) \times (s-k_2+1) \times k_0(b-k_3+1)$其中$k_0$是卷积核个数

## step2:高斯加权特征令牌化器
<img src="屏幕截图 2025-07-16 223904.png" width="300px"><br>
- 两层卷积层生成的特征图无法充分描述地物特征,于是将特征图定义为语义标记.
- 原输出图为$X\in \mathbb{R}^{u\times v \times z}$
- 输入高斯加权特征化令牌器的扁平化特征图为$X \in \mathbb{R}^{uv\times z}$
- 其中,uv是空间维度,z是光谱维度
- 特征标记被定义为T,T由如下运算得到
$$T=\underbrace{softmax(XW_a)^T}_AX $$

<font color="red">Q: $ W_a $ 是什么,怎么生成的,作用是什么 </font>
<font color="green">A:$w_a$是以高斯分布进行初始化的,其作用是1.将特征图映射到语义组2.增强样本可分性</font>

## step3:transformer编码器模块
<img src="屏幕截图 2025-07-16 224221.png" width="300px"><br>
### 1.输入处理:语义token与位置嵌入
- 模块的输入是高斯加权特征令牌化器生成的$T \in \mathbb{R}^{\omega \times z} $
- 为实现最终分类,在语义token前添加可以学习的分类token $T^{cls}_0$
- 位置嵌入:添加位置编码表述空间关系
### 2.多头自注意力机制与MLP
- 自注意力:计算qkv矩阵以捕捉token之间的依赖关系
- 多头自注意力机制:将QKV通过不同的权重矩阵进行SA,可以获得在不同语义之间的关联关系
- 对于MSA的输出,将其输入全连接层对于其输出的序列$T_{out}$,取第一个token作为分类特征向量(对应$T_0^{cls}$)